<a href="https://colab.research.google.com/github/guilhermehiroshifujii/Imersao_IA_05_2025/blob/main/Copy_of_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents= "Quando é a próxima Imersão IA com Google Gemini da Alura?"
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será lançada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn e outras plataformas onde a Alura costuma divulgar seus eventos e cursos.
*   **Assinar a newsletter da Alura:** Assim você receberá novidades e anúncios diretamente no seu e-mail.
*   **Verificar a página de cursos da Alura:** Consulte regularmente a página de cursos de Inteligência Artificial da Alura para ver se a Imersão IA com Google Gemini foi adicionada ao catálogo.

Além disso, você pode entrar em contato diretamente com a Alura através do site deles para perguntar sobre a previsão da próxima edição.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}

)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura ocorreu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini Alura', 'Alura Imersão IA Google Gemini']
Páginas utilizadas na resposta: starten.tech, tecmundo.com.br, youtube.com, alura.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

  buscador = Agent(
    name="agente_buscador",
    model="gemini-2.0-flash",
    description="Agente que buscas informações no google",
    tools=[google_search],
    instruction="""
    Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
    Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
    Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim e pode ser substituído por outro que tenha mais.
    Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
    """
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos


In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
         Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles. Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha, vamos criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n-- 📝 Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n-- 📝 Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------")

  entrada_do_agente_redator = agente_redator(topico, plano_de_post)
  print("\n-- 📝 Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(entrada_do_agente_redator))
  print("--------------------------------------------------------------")

  entrada_do_agente_revisor = agente_revisor(topico, entrada_do_agente_redator)
  print("\n-- 📝 Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(entrada_do_agente_revisor))
  print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Fiat Elba
Maravilha, vamos criar o post sobre novidades em Fiat Elba

-- 📝 Resultado do Agente 1 (Buscador) ---



> Para te ajudar a encontrar informações relevantes sobre o Fiat Elba, preciso primeiro entender o que você procura. Você está interessado em saber sobre:
> 
> *   **Relançamento do Fiat Elba?** Existe alguma notícia sobre a volta do modelo?
> *   **Notícias sobre o Fiat Elba Clássico?** Histórias, restaurações, ou eventos relacionados ao modelo original?
> *   **Elba como carro clássico/vintage?** Informações sobre o valor do carro, procura por ele, etc.
> 
> Com isso em mente, farei algumas pesquisas para encontrar as notícias mais recentes e relevantes sobre o Fiat Elba.
> 
> 
> Com base nas minhas buscas, aqui estão os lançamentos e notícias mais relevantes sobre o Fiat Elba nos últimos meses:
> 
> **1. Relançamento do Fiat Elba (Especulação e Humor):**
> 
> *   Há algumas notícias de abril de 2025 que mencionam um possível relançamento do Fiat Elba, impulsionado pela prisão de Fernando Collor. No entanto, essas notícias são de natureza satírica e humorística, como a da Revista Piauí ([10]). Não há informações concretas sobre um relançamento oficial por parte da Fiat.
> *   O site Garagem Master ([5], [8]) criou projeções de como seria um novo Fiat Elba Visione 2026 e Elba Stellare 2026, com designs modernos, mas mantendo a essência do clássico. É importante notar que são apenas exercícios de design.
> 
> **2. Fiat Elba como Símbolo Político:**
> 
> *   O Fiat Elba continua sendo lembrado como um símbolo do escândalo político envolvendo o ex-presidente Fernando Collor. Artigos recentes ([2], [3], [4], [6]) relembram o caso e como o carro se tornou um ícone da história política brasileira.
> 
> **3. Problemas e Legado do Fiat Elba Clássico:**
> 
> *   Um artigo de março de 2025 no Agora Motor ([1]) detalha os principais problemas enfrentados pelo Fiat Elba ao longo dos anos, como a suspensão frágil, ferrugem e falhas no sistema elétrico. Apesar disso, o artigo ressalta que o Elba conquistou uma base de fãs por seu custo-benefício e facilidade de manutenção.
> *   Um vídeo de janeiro de 2025 no YouTube ([9]) discute a verdade por trás do fim do Fiat Elba, destacando sua durabilidade, economia e capacidade de carga, mas também mencionando a tendência à corrosão.
> 
> **4. Valorização e Restauração:**
> 
> *   Há menções sobre o valor de um Fiat Elba Weekend 1.5 ano 1991, que pode chegar a R$ 6.198,00 (em maio de 2025), dependendo do estado de conservação ([2]).
> *   O Mercado Livre ([12]) oferece adesivos e emblemas para restauração do Fiat Elba.
> 
> **Em resumo:**
> 
> Não há um relançamento oficial do Fiat Elba. As notícias mais recentes giram em torno do seu legado como carro clássico, seu papel na história política brasileira e projetos de design que reimaginam o modelo para o futuro.
> 


--------------------------------------------------------------

-- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nas informações coletadas, o tema mais relevante para um post sobre o Fiat Elba é:
> 
> **O Fiat Elba: Um Clássico Brasileiro Entre a Nostalgia e a História Política**
> 
> **Pontos Relevantes:**
> 
> *   **Legado Histórico-Político:** O Fiat Elba como símbolo do governo Collor e do confisco de bens.
> *   **Nostalgia e Valorização:** O interesse crescente pelo Elba como carro clássico, restauração e valor de mercado.
> *   **Problemas e Qualidades:** Os defeitos conhecidos do modelo original (suspensão, ferrugem, elétrica) versus sua durabilidade, economia e espaço.
> *   **Especulações e Humor:** As notícias satíricas sobre um possível relançamento e as projeções de design moderno.
> 
> **Plano para o Post:**
> 
> 1.  **Introdução:**
>     *   Apresentar o Fiat Elba como um carro que marcou época no Brasil.
>     *   Destacar a dualidade do modelo: carro popular e símbolo político.
>     *   Mencionar o interesse atual pelo Elba como carro clássico.
> 2.  **O Elba e o Governo Collor:**
>     *   Explicar o contexto histórico do confisco de bens e como o Elba se tornou um símbolo desse período.
>     *   Incluir fotos e referências a reportagens da época.
> 3.  **Problemas e Qualidades do Fiat Elba:**
>     *   Listar os principais problemas enfrentados pelo modelo (suspensão, ferrugem, elétrica).
>     *   Destacar suas qualidades: durabilidade, economia, espaço interno e facilidade de manutenção.
>     *   Usar depoimentos de proprietários e mecânicos para enriquecer a discussão.
> 4.  **O Elba como Carro Clássico:**
>     *   Abordar o interesse crescente pelo Elba como carro clássico e de coleção.
>     *   Mostrar exemplos de restaurações e projetos de personalização.
>     *   Discutir o valor de mercado do Elba e a procura por peças de reposição.
> 5.  **Especulações e Humor:**
>     *   Mencionar as notícias satíricas sobre um possível relançamento do Elba.
>     *   Apresentar as projeções de design moderno (Elba Visione e Elba Stellare) como exercício de criatividade.
> 6.  **Conclusão:**
>     *   Reforçar o legado do Fiat Elba como um carro que faz parte da história do Brasil.
>     *   Convidar os leitores a compartilhar suas experiências e opiniões sobre o modelo.
>     *   Incentivar a discussão sobre o futuro do Elba como carro clássico.
> 
> Para aprofundar, posso pesquisar mais sobre:
> 
> *   A história do Fiat Elba no Brasil e no mundo.
> *   O mercado de carros clássicos no Brasil e o valor do Elba.
> *   Dicas de restauração e manutenção do Fiat Elba.
> *   Comparações entre o Fiat Elba e outros carros da época.
> 
> 
> Excelente! Com base nas informações adicionais, posso refinar o plano para o post sobre o Fiat Elba:
> 
> **Título:** Fiat Elba: Da Perua Popular ao Símbolo de um Escândalo - Uma História Brasileira
> 
> **Estrutura do Post:**
> 
> 1.  **Introdução:**
>     *   Apresentar o Fiat Elba como um carro que marcou época no Brasil, lançado em 1986 como a versão perua do Fiat Uno (Fontes: \[1], \[2], \[3], \[4], \[7]).
>     *   Destacar a dualidade do modelo: carro popular e símbolo político do impeachment de Collor (Fontes: \[2], \[4], \[5]).
>     *   Mencionar o interesse atual pelo Elba como carro clássico, com valor de mercado em torno de R$ 6.198,00 (em maio de 2025) para um modelo 1991 em bom estado (Fonte: \[5]).
> 2.  **O Elba e o Governo Collor:**
>     *   Explicar o contexto histórico do confisco de bens e como um Fiat Elba Weekend 1.5 1991 se tornou um símbolo do escândalo, sendo adquirido com recursos de contas fantasmas de PC Farias (Fontes: \[2], \[4], \[5]).
>     *   Incluir fotos e referências a reportagens da época, mostrando o carro como "prova material" no processo de impeachment (Fonte: \[4]).
> 3.  **Problemas e Qualidades do Fiat Elba:**
>     *   Listar os principais problemas enfrentados pelo modelo: suspensão frágil, ferrugem, falhas no sistema elétrico, desgaste rápido do motor e pneus (Fontes: \[1], \[11], \[15]).
>     *   Destacar suas qualidades: durabilidade, economia (média de 10 km/l na cidade e 14 km/l na estrada), espaço interno (maior espaço para bagagem de uma perua nacional na época, com 847 litros até o teto) e facilidade de manutenção (Fontes: \[1], \[7], \[11], \[15]).
>     *   Usar depoimentos de proprietários e mecânicos para enriquecer a discussão.
> 4.  **O Elba como Carro Clássico:**
>     *   Abordar o interesse crescente pelo Elba como carro clássico e de coleção, com comunidades online e clubes dedicados ao modelo.
>     *   Mostrar exemplos de restaurações e projetos de personalização (Fonte: \[20]).
>     *   Discutir o valor de mercado do Elba e a procura por peças de reposição (Fonte: \[5]).
>     *   Mencionar a oferta de adesivos e emblemas para restauração no Mercado Livre (Fonte: \[12]).
> 5.  **Comparativo com Concorrentes:**
>     *   Comparar o Fiat Elba com outros modelos da época, como Chevrolet Ipanema/Suprema, Ford Royale e VW Parati/Quantum, destacando suas vantagens e desvantagens em termos de espaço, desempenho e conforto (Fontes: \[10], \[13], \[14]).
>     *   Mencionar o comparativo da Revista Quatro Rodas em 1993, que avaliou as principais peruas nacionais (Fontes: \[10], \[13]).
> 6.  **Especulações e Humor:**
>     *   Mencionar as notícias satíricas sobre um possível relançamento do Elba, impulsionadas pela prisão de Fernando Collor (Fonte: \[10]).
>     *   Apresentar as projeções de design moderno (Elba Visione e Elba Stellare) como exercício de criatividade (Fontes: \[5], \[8]).
> 7.  **Conclusão:**
>     *   Reforçar o legado do Fiat Elba como um carro que faz parte da história do Brasil, tanto pela sua importância no mercado automotivo quanto pelo seu papel no cenário político.
>     *   Convidar os leitores a compartilhar suas experiências e opiniões sobre o modelo.
>     *   Incentivar a discussão sobre o futuro do Elba como carro clássico e a valorização da sua história.
> 
> Com este plano detalhado, o post terá uma estrutura sólida e abordará todos os pontos relevantes sobre o Fiat Elba, desde sua história e características até seu legado como carro clássico e símbolo político.
> 


--------------------------------------------------------------

-- 📝 Resultado do Agente 3 (Redator) ---



> **Rascunho do Post para Instagram:**
> 
> 🚗💨 **Fiat Elba: Mais que uma perua, um capítulo da história do Brasil!** 🇧🇷
> 
> Lançada como a versão familiar do Uno, a Elba logo conquistou as ruas... e se tornou um símbolo de uma época turbulenta. 🗓️ Quem lembra do confisco de bens e de um certo Elba Weekend 1.5 no meio do escândalo? 🙊
> 
> Mas nem só de polêmica viveu a Elba! Ela tinha seus encantos: espaçosa, econômica e fácil de manter. 💪 Claro, a suspensão pedia arrego e a ferrugem era inimiga, mas quem não tem seus defeitos, né? 😉
> 
> Hoje, a Elba virou peça de colecionador! 🤩 Tem gente restaurando, personalizando e mostrando que a perua tem muito a oferecer. E aí, qual a sua história com a Elba? Conta pra gente nos comentários! 👇
> 
> \#FiatElba \#CarrosAntigos \#Nostalgia \#Alura
> 


--------------------------------------------------------------

-- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!
> 


--------------------------------------------------------------
